<a href="https://colab.research.google.com/github/NeveykoArt/MachinLearning/blob/main/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

подключение модулей

In [15]:
import numpy as np
import pandas as pd
import csv

import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from google.colab import files
uploaded = files.upload()

Saving Fish.csv to Fish (1).csv


Загрузка датасета, нормализация

In [19]:
fish_market = pd.read_csv('Fish.csv')
fish_market.head(5)

numerics = ['int64', 'float64']
new_fish_market = fish_market.select_dtypes(include = numerics)

scaler = MinMaxScaler()
scaler.fit(new_fish_market)
scaled_features = scaler.transform(new_fish_market)
fish_market_MinMax = pd.DataFrame(data = scaled_features, 
columns = ["Weight", 
          "Length1",
          "Length2",
          "Length3",
          "Height", 
          "Width"])

fish_market_MinMax.head()

,Weight,Length1,Length2,Length3,Height,Width
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.063325,0.054054,0.057692,0.072727,0.129084,0.104627
2,0.129288,0.047297,0.070513,0.066667,0.115342,0.247683
3,0.159631,0.209459,0.230769,0.212121,0.162700,0.159541
4,0.248021,0.222973,0.230769,0.242424,0.124244,0.408103


In [22]:
X = fish_market_MinMax[['Length1', 'Length2', 'Length3', 'Height', 'Width']].values
y = fish_market_MinMax['Weight'].values

In [23]:
display(X, y)
X.shape, y.shape

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.05405405, 0.05769231, 0.07272727, 0.12908431, 0.10462688],
       [0.0472973 , 0.07051282, 0.06666667, 0.11534221, 0.2476829 ],
       [0.20945946, 0.23076923, 0.21212121, 0.16270001, 0.15954134],
       [0.22297297, 0.23076923, 0.24242424, 0.12424365, 0.40810345],
       [0.24324324, 0.27564103, 0.28484848, 0.28000538, 0.33241748],
       [0.24324324, 0.27564103, 0.27272727, 0.35760387, 0.46103967],
       [0.2972973 , 0.29487179, 0.3030303 , 0.15463224, 0.24544822],
       [0.2972973 , 0.29487179, 0.30909091, 0.33412666, 0.30179141],
       [0.35810811, 0.33974359, 0.37575758, 0.36393707, 0.34414038],
       [0.35135135, 0.35897436, 0.37575758, 0.36880463, 0.3971865 ],
       [0.37162162, 0.35897436, 0.37575758, 0.38340729, 0.29109426],
       [0.39864865, 0.39102564, 0.38787879, 0.30108915, 0.12748654],
       [0.42567568, 0.42307692, 0.44242424, 0.32175608, 0.38568341],
       [0.41891892, 0.42307692, 0.

array([0.        , 0.06332454, 0.1292876 , 0.15963061, 0.24802111,
       0.27440633, 0.34036939, 0.19525066, 0.27440633, 0.34036939,
       0.30738786, 0.34036939, 0.34036939, 0.1292876 , 0.47229551,
       0.47229551, 0.60422164, 0.60422164, 0.48548813, 0.53825858,
       0.43931398, 0.58443272, 0.49868074, 0.57783641, 0.60422164,
       0.63720317, 0.63060686, 0.62269129, 0.80211082, 1.        ,
       0.8944591 , 0.94063325, 0.90105541, 0.96701847, 0.93403694])

((35, 5), (35,))

функция потерь, градиентный спуск

In [32]:
def calculate_matrix_loss(X: np.ndarray, w: np.ndarray, y: np.ndarray) -> float:
        return np.sum((y - (X @ w)) ** 2) / 2

class GradientDescent:
    def __init__(self, dimension: int, learning_rate: float = 1e-2, iteration: int = 10000, eps: float = 1e-4) -> None:
        self.w: np.ndarray = np.zeros(dimension)
        self.learning_rate: float = learning_rate
        self.iteration: int = iteration
        self.eps: float = eps

    def calculate_gradient(self, X: np.ndarray, y: np.ndarray) -> float:
        return (y - (X @ self.w)) @ (-X)

    def update_weights(self, X: np.ndarray, y: np.ndarray) -> np.ndarray:
        return self.w - self.learning_rate * self.calculate_gradient(X, y)

    def run(self, X: np.ndarray, y: np.ndarray) -> tuple:
        # matrix_loss = None
        next_w = self.w
        logs = pd.DataFrame(columns=['current-weights', 'next-weights', 'loss'])
        for i in np.arange(self.iteration):
            self.w = next_w
            next_w = self.update_weights(X, y)
            # if calculate_matrix_loss(X, next_w, y) <= self.eps:
            #     break
            log = pd.DataFrame([[self.w, next_w, calculate_matrix_loss(X, self.w, y)]], columns=['current-weights', 'next-weights', 'loss'])
            logs = pd.concat([logs, log], sort=False, ignore_index=True)
            if np.linalg.norm(self.w - next_w, ord=2) <= self.eps:
                break
        return (self.w, logs)

линейная регрессия

In [33]:
class LinearRegression:
    def __init__(self, iteration: int = 10000, eps: float = 1e-4) -> None:
        self.iteration: int = iteration
        self.eps: float = eps
        self.w = None
        self.logs = None
        self.loss: float = 0

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        # add a dummy feature
        X = np.column_stack([X, np.ones(X.shape[0])])
        descent = GradientDescent(dimension=X.shape[1], learning_rate=1e-2, iteration=self.iteration, eps=self.eps)
        self.w, self.logs = descent.run(X, y)
        self.loss = calculate_matrix_loss(X, self.w, y)
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        return X @ self.w[:-1] + self.w[-1]

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
regressor = LinearRegression()
regressor.fit(X_train, y_train)
regressor.logs.head(50)

,current-weights,next-weights,loss
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.08180400057049132, 0.08442087815438738, 0.0...",4.280145
1,"[0.08180400057049132, 0.08442087815438738, 0.0...","[0.1120387023359479, 0.11564325654150584, 0.11...",0.693255
2,"[0.1120387023359479, 0.11564325654150584, 0.11...","[0.12443325547570731, 0.12846126443751524, 0.1...",0.256147
3,"[0.12443325547570731, 0.12846126443751524, 0.1...","[0.13060236430151143, 0.13485635896286863, 0.1...",0.193281
4,"[0.13060236430151143, 0.13485635896286863, 0.1...","[0.13454781750247402, 0.13895671561980708, 0.1...",0.175666
5,"[0.13454781750247402, 0.13895671561980708, 0.1...","[0.1376505583216646, 0.14218691434115024, 0.14...",0.164240
6,"[0.1376505583216646, 0.14218691434115024, 0.14...","[0.1403891804942825, 0.145040693969178, 0.1471...",0.154308
7,"[0.1403891804942825, 0.145040693969178, 0.1471...","[0.14293151584941152, 0.1476912182536843, 0.14...",0.145251
8,"[0.14293151584941152, 0.1476912182536843, 0.14...","[0.14533829030811868, 0.15020116198811764, 0.1...",0.136939
9,"[0.14533829030811868, 0.15020116198811764, 0.1...","[0.14763323654802157, 0.15259503741467026, 0.1...",0.129304


In [35]:
print(f"Prediction: {regressor.predict(X_test)}\nOriginal: {y_test}\nLoss: {regressor.loss}")

Prediction: [0.65916493 0.38770255 0.61968319 0.56325418 0.48430873 0.86747456
 0.5038455 ]
Original: [0.63060686 0.1292876  0.60422164 0.58443272 0.47229551 1.
 0.53825858]
Loss: 0.041213084675696066


array([ 0.16781137,  0.17947779,  0.18466662,  0.32441018,  0.17401128,
       -0.01285879])